# LSTM-VQE for Molecule

In [ ]:
!pip install pennylane
!pip install aiohttp fsspec h5py
!pip install pennylane-lightning
!python -m pip install pennylane pennylane-lightning pennylane-lightning-gpu --upgrade
!pip install custatevec-cu12

In [ ]:
import numpy as np
import pandas as pd
import pennylane as qml
from pennylane import qchem
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.optimize import minimize
import optax
from jax import numpy as np
import jax
import time

import os
import logging

# 設置靜音等級
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 僅顯示錯誤，忽略警告
tf.get_logger().setLevel(logging.ERROR)

## Loaded data <a name="loadeddata"></a>

In [ ]:
def load_data(molecule, basis="STO-3G"):
  data_list = []

  for mol,bondlengths in molecule:
    # If bondlengths is None, load default bond length
    if bondlengths is None:
      data = qml.data.load("qchem", molname=mol, basis=basis)

    else:
      # Otherwise, iterate over the bond lengths
      data = qml.data.load("qchem", molname=mol, basis=basis, bondlength=bondlengths)

    for entry in data:
      data_list.append(entry)

  df = pd.DataFrame(data_list)
  return df

# Input molecule data
# HeH2 only has 6-31G, also, C2H6, CO2, N2H4, C2H4, N2H2, CH2O, C2H2 no data
# molecule = ["LiH", "Li2", "N2", "CO2", "BH3", "H2O2", "O2", "CO", "C2", "H4", "BeH2", "H3+", "NH3", "H2O",
            #"CH4", "HeH+", "H8", "H10", "NeH+", "OH-", "H6", "HF", "H2"]

molecule = [
    ("H2", None),
    ("H4", None),
    ("H6", None)
]

# Load dataset
dataset = load_data(molecule, basis="STO-3G")
print(dataset.head())

## Proprocess <a name="proprocess"></a>

In [ ]:
max_qubits = 0

for i in range(len(dataset[0])):
  if len(dataset[0][i].hf_state) > max_qubits:
    max_qubits = len(dataset[0][i].hf_state)

print(max_qubits)

In [ ]:
def electrons_and_orbitals(hf_state):
  electrons = sum(hf_state)
  orbitals = int(len(hf_state))

  return electrons, orbitals

In [ ]:
def max_params (dataset):
  max_s_wires = 0
  max_d_wires = 0

  for data in dataset:
    electrons, orbitals = electrons_and_orbitals(data.hf_state)

    single, double = qchem.excitations(electrons, orbitals)
    s_w, d_w = qml.qchem.excitations_to_wires(single, double)
    single = len(s_w)
    double = len(d_w)
    max_s_wires = max(max_s_wires, single)
    max_d_wires = max(max_d_wires, double)

  return max_s_wires, max_d_wires

max_s_params, max_d_params = max_params(dataset[0])
print(f"the number of max single wires: {max_s_params}")
print(f"the number of max double wires: {max_d_params}")

In [ ]:
# for test

dataset_list = list(dataset[0])
train_set = dataset_list[:len(molecule)-1]
test_set = dataset_list[len(molecule)-1]
print(f"train set: {train_set}")
print(f"test set: {test_set}")

## Model <a name="model"></a>

In [ ]:
def vqe_molecule_u(data,max_qubits):

  # Number of qubits (wires) equal to the max qubits
  wires = range(max_qubits)
  # Number of qubits (wires) equal to the number of qubits required for the HF state
  orbitals = len(data.hf_state)
  # the number of electrons and molecules
  electrons = sum(data.hf_state)
  # the wires of single and double excitations
  single_wires, double_wires = qchem.excitations(electrons,orbitals)
  s_wires, d_wires = qml.qchem.excitations_to_wires(single_wires, double_wires)

  num_single = len(d_wires)
  num_double = len(s_wires)

  # Create the actual quantum circuit for the VQE algorithm
  def circuit(params, wires):

    # the params for single and double excitations
    params = tf.reshape(params, (max_s_params + max_d_params,))

    single_params = params[:num_single]

    double_params = params[max_s_params: max_s_params + num_double]

    initial_params_u = tf.concat([single_params, double_params], axis=0)

    qml.UCCSD(initial_params_u, range(orbitals), s_wires, d_wires, data.hf_state)
    # # something we want to train, ansatz
    # if ansatzs == "UCCSD":
    #   qml.UCCSD(initial_params_u, range(orbitals), s_wires, d_wires, data.hf_state)
    # elif ansatzs == "StronglyEntangling":
    #   qml.templates.StronglyEntanglingLayers(weights = params, wires = wires)
    # elif ansatzs == "HEA":
    #   qml.HEA(initial_params_u, range(orbitals), s_wires, d_wires, data.hf_state)

    return qml.expval(data.hamiltonian)

  # Evaluates the cost Hamiltonian
  def hamiltonian_cost(params, **kwargs):
    # Define device
    # This qnode evaluates the expectation value of the cost hamiltonian operator
    dev = qml.device("lightning.qubit", wires=wires)
    cost = qml.QNode(circuit, dev, diff_method="adjoint", interface="tf")

    return cost(params, wires = wires)

  return hamiltonian_cost

In [ ]:
def vqe_optimization(molecule_cost, initial_params, max_iter=10, learning_rate=0.1, conv_tol = 1e-6):

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # opt = qml.AdamOptimizer(stepsize=learning_rate)

    # 初始化变量来存储迭代信息
    params = tf.Variable(initial_params)
    energy_history = [molecule_cost(params)]
    param_history = [params]
    conv_iter = max_iter

    # 优化循环
    for iteration in range(max_iter):
        # 计算梯度并更新参数
        with tf.GradientTape() as tape:
            cost = molecule_cost(params)
        gradient = tape.gradient(cost, params)

        opt.apply_gradients(zip([gradient],[params]))
        param_history.append(params)
        energy_history.append(molecule_cost(params))

        conv = np.abs(energy_history[-1].numpy() - energy_history[-2].numpy())

        # 打印当前迭代的参数和能量值
        if iteration%10 == 0:
          print(f"Step =  {iteration+1}/{max_iter}, Energy= {energy_history[-1]:.8f} Ha")

        if conv <= conv_tol:
          conv_iter = iteration+1
          break

    # 返回优化后的参数和能量值，以及每次迭代的记录
    return conv_iter, param_history[-1], energy_history[-1], param_history, energy_history

In [ ]:
param_dim_u = max_s_params + max_d_params
cell_units_u = param_dim_u
cell_u = tf.keras.layers.LSTMCell(cell_units_u)
model_u = tf.keras.Sequential([cell_u])

In [ ]:
# 全局定義
param_dim_u = max_s_params + max_d_params
cell_units_u = param_dim_u
cell_u = tf.keras.layers.LSTMCell(cell_units_u)
# 構建 cell_u
input_shape = (None, 1 + param_dim_u)
cell_u.build(input_shape)

In [ ]:
initial_h_u = tf.zeros(shape=(1, cell_units_u), dtype=tf.float32)
initial_c_u = tf.zeros(shape=(1, cell_units_u), dtype=tf.float32)
initial_cost_u = tf.zeros(shape=(1, 1), dtype=tf.float32)
initial_params_u = tf.zeros(shape=(1,param_dim_u), dtype=tf.float32)

In [ ]:
molecule_cost_list_u = [vqe_molecule_u(m, max_qubits) for m in train_set]

In [ ]:
def rnn_iteration_u(inputs, molecule_cost, param_dim):
    prev_cost, prev_params, prev_h, prev_c = inputs
    new_input = tf.keras.layers.concatenate([prev_cost, prev_params])
    new_params, [new_h, new_c] = cell_u(new_input, states=[prev_h, prev_c])
    _params = tf.reshape(new_params, [param_dim])
    _cost = molecule_cost(_params)
    new_cost = tf.reshape(tf.cast(_cost, dtype=tf.float32), shape=(1, 1))
    return [new_cost, new_params, new_h, new_c]

In [ ]:
def recurrent_loop_u(molecule_cost, param_dim = param_dim_u, steps = 5, intermediate_steps=False):
    """Creates the recurrent loop for the Recurrent Neural Network."""

    initial_cost = tf.zeros(shape=(1, 1), dtype=tf.float32)
    initial_params = tf.zeros((1, param_dim), dtype=tf.float32)
    initial_h = tf.zeros(shape=(1, cell_u.units), dtype=tf.float32)
    initial_c = tf.zeros(shape=(1, cell_u.units), dtype=tf.float32)

    inputs = [initial_cost, initial_params, initial_h, initial_c]
    param_outputs = []
    cost_outputs = []


    for _ in range(steps):
      inputs = rnn_iteration_u(inputs, molecule_cost, param_dim)
      param_outputs.append(inputs[1])
      cost_outputs.append(inputs[0])



    #min_cost = float('inf')
    #min_cost_j_list = []
    loss = 0
    min_cost = cost_outputs[0]

    for t in range(1,len(cost_outputs)):
      min_cost = tf.minimum(min_cost, cost_outputs[t-1]) #t and max(next line) is good too
      loss += tf.maximum(0.0, cost_outputs[t] - min_cost)
      #loss += tf.maximum(0.0, cost_outputs[t]-min_cost)
    #  min_cost_j = cost_outputs[t-1]
    #  for j in range(t):
    #    if cost_outputs[j] < min_cost_j:
    #      min_cost_j = cost_outputs[j]
    #  loss += (cost_outputs[t] - min_cost_j)

    if intermediate_steps:
        return param_outputs + [loss]
    else:
        return loss

In [ ]:
def train_step_u(molecule_cost, param_dim = param_dim_u, steps = 10):
    """Single optimization step in the training procedure."""

    with tf.GradientTape() as tape:
        loss = tf.cast(recurrent_loop_u(molecule_cost=molecule_cost, param_dim=param_dim, steps = steps), dtype=tf.float32)
    # Evaluates gradients, cell is the LSTM cell defined previously
    grads = tape.gradient(loss, cell_u.trainable_weights)

    #print("Gradients:", grads)

    # Apply gradients and update the weights of the LSTM cell
    opt_u.apply_gradients(zip(grads, cell_u.trainable_weights))

    return loss

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100,
    decay_rate=0.9,
    staircase=True
)

opt_u = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Set the number of training epochs
epochs_u = 50
steps_per_epoch_u = 5

previous_mean_loss_u = None

for epoch in range(epochs_u):

    start_time = time.time()

    if epoch % 10 == 0:
      print(f"Epoch {epoch+1}/{epochs_u}")

    epoch_loss_u = []
    for i, molecule_cost in enumerate(molecule_cost_list_u):
        loss_u = train_step_u(molecule_cost, param_dim_u, steps = steps_per_epoch_u)
        epoch_loss_u.append(loss_u.numpy())

        # Log every 5 batches.
        if i % 10 == 0:
            print(f" > Molecule {i+1}/{len(molecule_cost_list_u)} - Loss: {loss_u.numpy()[0][0]:.8f}") #[0][0]
            print(f" > steps_per_epoch: {steps_per_epoch_u}")

    epoch_loss_u = np.array(epoch_loss_u)
    mean_loss_u = np.mean(epoch_loss_u)

    if epoch % 10 == 0:
      print(f" > Mean loss: {mean_loss_u}")

    if mean_loss_u > 1e-2:
      steps_per_epoch_u = min(steps_per_epoch_u + 2, 15)
    elif mean_loss_u < 1e-3:
      steps_per_epoch_u = max(steps_per_epoch_u -2, 5)

    #if epoch > 0 and abs(previous_mean_loss - mean_loss) < 1e-4:
     # print(f"Training Converged at Epoch {epoch+1}")
      #break

    if previous_mean_loss_u is not None:
        rel_change_u = abs(previous_mean_loss_u - mean_loss_u) / abs(previous_mean_loss_u)
        if rel_change_u < 1e-4:
            print(f"Training Converged at Epoch {epoch+1}")
            break

    # Update the previous mean loss
    previous_mean_loss_u = mean_loss_u

    end_time = time.time()
    execution_time = end_time - start_time


    print(f"Execution time of train for {epoch+1}: {execution_time:.1f} seconds")

### Traditional VQE <a name="vqe"></a>

In [ ]:
def vqe_optimization_traditional(molecule_cost, initial_params, max_iter=10, learning_rate=0.1, conv_tol = 1e-6):

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # 初始化变量来存储迭代信息
    params = tf.Variable(initial_params)
    energy_history = [molecule_cost(params)]
    param_history = [params]
    conv_iter = max_iter

    # 优化循环
    for iteration in range(max_iter):
        # 计算梯度并更新参数
        with tf.GradientTape() as tape:
            cost = molecule_cost(params)
        gradient = tape.gradient(cost, params)

        opt.apply_gradients(zip([gradient],[params]))
        param_history.append(params)
        energy_history.append(molecule_cost(params))

        conv = np.abs(energy_history[-1].numpy() - energy_history[-2].numpy())

        # 打印当前迭代的参数和能量值
        if iteration%10 == 0:
          print(f"Step =  {iteration+1}/{max_iter}, Energy= {energy_history[-1]:.8f} Ha")

        if conv <= conv_tol:
          conv_iter = iteration+1
          break

    # 返回优化后的参数和能量值，以及每次迭代的记录
    return conv_iter, param_history[-1], energy_history[-1], param_history, energy_history

## Test <a name="test"></a>
### LSTM-VQE (UCCSD ansatz)

In [ ]:
test_molecule_cost_u = vqe_molecule_u(test_set, max_qubits)

initial_h_u = tf.zeros(shape=(1, cell_units_u), dtype=tf.float32)
initial_c_u = tf.zeros(shape=(1, cell_units_u), dtype=tf.float32)
initial_cost_u = tf.zeros(shape=(1, 1), dtype=tf.float32)
initial_params_u = tf.zeros(shape=(1,param_dim_u), dtype=tf.float32)

# 模型推理，獲得參數
test_output_u = rnn_iteration_u([initial_cost_u, initial_params_u, initial_h_u, initial_c_u],
                            test_molecule_cost_u,
                            param_dim = param_dim_u)


lstm_initial_params_u = tf.reshape(test_output_u[1], shape=(max_s_params+max_d_params)).numpy()

conv_iter_u, lstm_optimized_params_u, lstm_optimized_energy_u, lstm_param_history_u, lstm_energy_history_u = vqe_optimization(
    test_molecule_cost_u,
    initial_params=lstm_initial_params_u,
    max_iter=100,
    learning_rate=0.1,
    conv_tol = 1e-6
)

Step =  1/100, Energy= -2.89416718 Ha
Step =  11/100, Energy= -3.18711448 Ha
Step =  21/100, Energy= -3.19362712 Ha
Step =  31/100, Energy= -3.20309043 Ha
Step =  41/100, Energy= -3.20643759 Ha
Step =  51/100, Energy= -3.20800400 Ha
Step =  61/100, Energy= -3.20852470 Ha
Step =  71/100, Energy= -3.20868826 Ha


### Traditional VQE <a name="test_traditional"></a>

In [ ]:
molecule_cost = vqe_molecule_u(test_set, max_qubits)
# create  weights

initial_params = tf.zeros(shape=(max_s_params+max_d_params,), dtype = tf.float32)
# VQE
conv_iter, optimized_params, optimized_energy, vqe_param_history, vqe_energy_history = vqe_optimization_traditional(
    molecule_cost,
    initial_params,
    max_iter=100,
    learning_rate=0.1,
    conv_tol = 1e-6
)

# 输出最终优化结果
print("Optimized Parameters:", optimized_params)
print("Optimized Cost (Energy):", optimized_energy)

## Result <a name="result"></a>

In [ ]:
print("=== Comparison of VQE Optimization ===")
print("Real data")
print(f"  Molecule: {test_set.molecule}")
print(f"  VQE energy: {test_set.vqe_energy}")
print(f"  FCI energy: {test_set.fci_energy}")
print(f"  erros(%):{abs((test_set.vqe_energy-test_set.fci_energy)/test_set.fci_energy*100)}%")
print("VQE")
error_vqe = optimized_energy - test_set.fci_energy
print(f"  Iter: {conv_iter}")
print(f"  Final Energy: {optimized_energy}")
print(f"  error:{error_vqe}")
print(f"  error(%): {abs(error_vqe/test_set.fci_energy*100)}%")

print("LSTM-VQE (UCCSD ansatz)")
error_lstm_u = lstm_optimized_energy_u - test_set.fci_energy
print(f"  Iter: {conv_iter_u}")
print(f"  Final Energy: {lstm_optimized_energy_u}")
print(f"  error:{error_lstm_u}")
print(f"  error(%): {abs(error_lstm_u/test_set.fci_energy*100)}%")

In [ ]:
print("=== Comparison of VQE Optimization ===")
print("Real data")
print(f"  Molecule: {test_set.molecule}")
print(f"  VQE energy: {test_set.vqe_energy}")
print(f"  FCI energy: {test_set.fci_energy}")
print(f"  erros(%):{abs((test_set.vqe_energy-test_set.fci_energy)/test_set.fci_energy*100)}%")
print("VQE")
error_vqe = optimized_energy - test_set.fci_energy
print(f"  Iter: {conv_iter}")
print(f"  Final Energy: {optimized_energy}")
print(f"  error:{error_vqe}")
print(f"  error(%): {abs(error_vqe/test_set.fci_energy*100)}%")

print("LSTM-VQE (UCCSD ansatz)")
error_lstm_u = lstm_optimized_energy_u - test_set.fci_energy
print(f"  Iter: {conv_iter_u}")
print(f"  Final Energy: {lstm_optimized_energy_u}")
print(f"  error:{error_lstm_u}")
print(f"  error(%): {abs(error_lstm_u/test_set.fci_energy*100)}%")

## Result (Loss) <a name="loss"></a>

In [ ]:
res = recurrent_loop_u(test_molecule_cost_u, steps=30, intermediate_steps=True)

# Extract all angle suggestions
start_zeros = tf.zeros(shape=(max_s_params+max_d_params))
guesses = [start_zeros] + [
    tf.reshape(r, shape=(max_s_params + max_d_params)) for r in res[:-1]
    ]


final_loss = res[-1]

lstm_losses_u = [
    test_molecule_cost_u(tf.reshape(guess, shape=(1, max_s_params + max_d_params)))
    for guess in guesses
]

fig, ax = plt.subplots()

plt.plot(lstm_losses_u, color="blue", lw=3, ls="-.", label="LSTM")

plt.grid(ls="--", lw=2, alpha=0.25)
plt.ylabel("Cost function", fontsize=12)
plt.xlabel("Iteration", fontsize=12)
plt.legend()
#ax.set_xticks([0, 5, 10, 15, 20])
ax.set_xticks(range(0, len(lstm_losses_u)))
plt.show()